In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import pyemu
import shutil
import geopandas as gpd
from pathlib import Path
from mf6rtm import utils, mup3d
from flopy.utils.gridintersect import GridIntersect
from flopy.utils import  cvfdutil
from vorflow import ConceptualMesh, MeshGenerator, VoronoiTessellator
from shapely import box, Point
from herebedragons import *
# from collections.abc import Iterable
# from pypestutils.pestutilslib import PestUtilsLib
# from collections import defaultdict
# lib = PestUtilsLib()

# Background

# Grid and domain

In [ ]:
domain = gpd.read_file(Path('data', 'domain.gpkg'))
Ly = domain.geometry.total_bounds[3] - domain.geometry.total_bounds[1]
Lx = domain.geometry.total_bounds[2] - domain.geometry.total_bounds[0]

xul = domain.geometry.total_bounds[0]
yul = domain.geometry.total_bounds[1]

domain_ext = box(xul, yul, xul + Lx, yul +  Ly)
domain_ext

In [ ]:
wells = pd.read_csv(Path('data', 'wells.csv'))
refinement = gpd.read_file(Path('data', 'refinement.gpkg'))

obsloc = pd.read_csv(Path('data', 'obs_loc.csv'))
obsloc.head()

wells

In [ ]:
from shapely.geometry import LineString

geom = refinement.geometry[0]
minx, miny, maxx, maxy = geom.bounds

# Create line from bottom-left to bottom-right of top edge
top_line = LineString([(minx, maxy), (maxx, maxy)])
bottom_line = LineString([(minx, miny), (maxx, miny)])

In [ ]:
background_lc = 25.0

blueprint = ConceptualMesh()
blueprint.add_polygon(domain_ext, zone_id=1)#,border_density=100,dist_max_in=background_lc)

blueprint.add_line(bottom_line, line_id="Refinement-Line",
                     resolution=5,
                        dist_max=1*background_lc,
                        # z_order=3
                        )

for wid in wells.index:
    blueprint.add_point(Point(wells.loc[wid, 'x'], wells.loc[wid, 'y']), point_id=f"{wells.loc[wid, 'name']}", 
                    resolution=2,
                    dist_max=1.5*background_lc)
    print(f"Added well {wells.loc[wid, 'name']} at ({wells.loc[wid, 'x']}, {wells.loc[wid, 'y']})")
clean_polys, clean_lines, clean_pts = blueprint.generate()

# 2. Mesh Generation
mesher = MeshGenerator(background_lc=background_lc, verbosity=0)
mesher.generate(clean_polys, clean_lines, clean_pts)

# 3. Voronoi Conversion
tessellator = VoronoiTessellator(mesher, blueprint, clip_to_boundary=True)
grid_gdf = tessellator.generate()

In [ ]:
grid_gdf

In [ ]:
fig, ax = plt.subplots(1, 1)

grid_gdf.plot(
    ax=ax,
    alpha=0.5,
    edgecolor='k',
    linewidth=0.2
)
ax.scatter(
    wells['x'],
    wells['y'],
    color='red',
    marker='.',
    s=100,
    label='Wells'
)
ax.legend()

ax.set_aspect('equal')
fig.tight_layout()

In [ ]:
from vorflow.utils import *

gdf = calculate_mesh_quality(grid_gdf,calc_ortho=True)
gdf.plot(column='drift_ratio', cmap='viridis', legend=True, figsize=(10,8),vmax=.25, aspect='equal')
summarize_quality(gdf)

grid_gdf.to_file(Path("data", "mf6_grid.shp"))

In [ ]:
verts, iverts = cvfdutil.shapefile_to_cvfd('data/mf6_grid.shp')
verts
# xcyc
gridprops = cvfdutil.get_disv_gridprops(verts, iverts, xcyc=None)
gridprops

In [ ]:
ws = Path("model")
if ws.exists():
    shutil.rmtree(ws)

flow_modelname = "gwf"
nper = 39  # Number of stress periods
perioddata= [(2, 2, 1), (4, 4, 1), (4, 4, 1), (4, 4, 1), (7, 7, 1),
            (7, 7, 1), (7, 7, 1), (7, 7, 1), (14, 14, 1), (14, 14, 1), 
            (15, 15, 1), (13, 13, 1), (14, 14, 1), (14, 14, 1), (14, 14, 1), 
            (21, 21, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
            (28, 28, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
            (35, 35, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
            (35, 35, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
            (28, 28, 1), (35, 35, 1), (35, 35, 1),(28, 28, 1)]


sim = flopy.mf6.MFSimulation(sim_name=f"{flow_modelname}", version='mf6', sim_ws=ws)
utils.prep_bins(ws, get_only=['mf6', 'libmf6', 'pestpp-ies', 'pestpp-mou'])
# ?utils.prep_bins

# specify tdis
tdis = flopy.mf6.ModflowTdis(sim, pname="tdis", time_units="DAYS", 
                                nper=nper, perioddata=perioddata)
outer_dvclose = 1e-7
inner_dvclose = 1e-7
ims = flopy.mf6.ModflowIms(sim, 
                        #    pname="ims", 
                        complexity="complex",
                        outer_dvclose=outer_dvclose,
                        inner_dvclose=inner_dvclose,
                        filename=f"{flow_modelname}.ims")
sim.register_ims_package(ims, 
                            [flow_modelname])

# start model build to refine 
model_nam_file = "{}.nam".format(flow_modelname)
gwf = flopy.mf6.ModflowGwf(sim, modelname=flow_modelname, 
                            model_nam_file=model_nam_file, exe_name='mf6')

disv = flopy.mf6.ModflowGwfdisv(gwf,
                                nlay=12,
                                ncpl=gridprops['ncpl'],
                                nvert=gridprops['nvert'],
                                vertices=gridprops['vertices'],
                                cell2d=gridprops['cell2d'],
                                top=-273.0,
                                botm=0.0,
                                filename=f"{flow_modelname}.disv")

botms = get_botms(gwf, ws)
disv.botm.set_data(botms)
disv.set_all_data_external()

nlay = disv.nlay.get_data()
ncpl = disv.ncpl.get_data()

ihead = 0 #(meters)
strt = ihead * np.ones((nlay, ncpl))
ic = flopy.mf6.ModflowGwfic(gwf, pname="ic", strt=strt)
ic.set_all_data_external()

npf = flopy.mf6.ModflowGwfnpf(
        gwf,
        icelltype=0,
        k=1,
        k33=1,
        # k33overk = True
    )
npf.set_all_data_external()

ss = np.ones((nlay, ncpl)) * 1.e-4
sto = flopy.mf6.ModflowGwfsto(gwf, 
                                ss=ss, 
                                iconvert=0,
                                # steady_state={0: False},
                                transient={0: True})
sto.set_all_data_external()

chd = make_chd(gwf, conservative_tracer='Cl', mup3d_m=None)
chd.export(os.path.join(ws, f"chd.vtk"), fmt="vtk")

# disv.export(os.path.join(ws, f"disv.vtk"), fmt="vtk")

    # create the output control
headfile = f"{flow_modelname}.hds"
head_filerecord = [headfile]
budgetfile = f"{flow_modelname}.cbb"
budget_filerecord = [budgetfile]
saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
printrecord = [("HEAD", "LAST")]

oc = flopy.mf6.ModflowGwfoc(
    gwf,
    saverecord=saverecord,
    head_filerecord=head_filerecord,
    budget_filerecord=budget_filerecord,
    printrecord=printrecord,)
sim.write_simulation()